## Import

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load

In [ ]:
# Load historical data
file_path_anleihen_etf = "./data/Anleihen und Indizes historische Zinsen.xlsx"
file_path_festgeld_tagesgeld = "./data/Handelsblatt_Spezial.xlsx"
file_path_inflation = "./data/Inflation_historisch_final.xlsx"

df_anleihen = pd.read_excel(file_path_anleihen_etf, sheet_name="Anleihen", skiprows=2)
df_etf = pd.read_excel(file_path_anleihen_etf, sheet_name="Indizes")
df_festgeld_tagesgeld= pd.read_excel(file_path_festgeld_tagesgeld, sheet_name="Monatswert", skiprows=1)
df_inflation= pd.read_excel(file_path_inflation, skiprows=2)

## Cleaning

In [ ]:
df_anleihen = df_anleihen.drop(df_anleihen.columns[[6, 11, 12, 13, 14, 15]], axis=1)
df_anleihen.columns = ["date", "Australien", "Kanada", "Frankreich", "Deutschland", "Japan", "Spanien", "Schweiz", "Großbritannien", "USA"]

df_etf.columns = ["date", "Dow Jones", "S&P-500", "S&P/TSX Composite", "CAC 40", "FTSE 100", "Bovespa", "Shanghai Composite", "Korea Composite", "NIKKEI 225", "IBEX 35", "S&P/ASX 50", "OMX Stockholm", "HANG SENG", "BSE SENSEX", "Dax", "FTSE All"]

df_festgeld_tagesgeld = df_festgeld_tagesgeld.drop(df_festgeld_tagesgeld.columns[[8]], axis=1)
df_festgeld_tagesgeld.columns = ["date", "Festgeld_05", "Festgeld_1", "Festgeld_2", "Festgeld_5", "Festgeld_10", "Tagesgeld", "Inflation_Deutschland"]
df_festgeld_tagesgeld = df_festgeld_tagesgeld.drop(columns=["Inflation_Deutschland"])

df_inflation = df_inflation.drop(df_inflation.columns[[6, 7, 8, 9]], axis=1)
df_inflation.columns = ["date", "Australien", "Kanada", "Deutschland", "Frankreich", "Japan", "Spanien", "Schweiz", "Großbritannien", "USA"]

df_anleihen.head(5)


,date,Australien,Kanada,Frankreich,Deutschland,Japan,Spanien,Schweiz,Großbritannien,USA
0,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.401667
1,1955,NaN,3.189167,NaN,NaN,NaN,NaN,2.982500,NaN,2.816667
2,1956,NaN,3.605833,NaN,NaN,NaN,NaN,3.125000,NaN,3.182500
3,1957,NaN,4.125000,NaN,7.516667,NaN,NaN,3.667500,NaN,3.647500
4,1958,NaN,4.115833,NaN,6.783333,NaN,NaN,3.153333,NaN,3.315833


##### Reduce Anleihen by inflation

In [ ]:
# Extract the year from the 'date' column
df_anleihen['date'] = df_anleihen['date'].astype(str).str.extract(r'(\d{4})')[0]
df_inflation['date'] = df_inflation['date'].astype(str).str.extract(r'(\d{4})')[0]

# Convert the extracted year to datetime format
df_anleihen['date'] = pd.to_datetime(df_anleihen['date'], format='%Y', errors='coerce')
df_inflation['date'] = pd.to_datetime(df_inflation['date'], format='%Y', errors='coerce')

# Drop rows with invalid dates
df_anleihen = df_anleihen.dropna(subset=['date'])
df_inflation = df_inflation.dropna(subset=['date'])

df_anleihen.set_index('date', inplace=True)
df_inflation.set_index('date', inplace=True)

# Reorder the columns in df_inflation to match the order of columns in df_anleihen
df_inflation = df_inflation[df_anleihen.columns]

# Subtract values in df_inflation from df_anleihen where both values exist
df_anleihen_inflation = df_anleihen.subtract(df_inflation, fill_value=np.nan)

# Reset the index to bring 'date' back as a column and convert it to year format
df_anleihen_inflation.reset_index(inplace=True)
df_anleihen_inflation['date'] = df_anleihen_inflation['date'].dt.year

,date,Australien,Kanada,Frankreich,Deutschland,Japan,Spanien,Schweiz,Großbritannien,USA
0,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.081667
5,1955,NaN,3.009167,NaN,NaN,NaN,NaN,2.052500,NaN,3.096667
6,1956,NaN,2.255833,NaN,NaN,NaN,NaN,1.615000,NaN,1.662500
7,1957,NaN,0.805000,NaN,5.516667,NaN,NaN,1.697500,NaN,0.307500
8,1958,NaN,1.745833,NaN,4.483333,NaN,NaN,1.313333,NaN,0.575833
9,1959,NaN,3.838333,NaN,5.166667,NaN,NaN,3.726667,NaN,3.323333


In [ ]:
# Convert the 'date' column to datetime format
df_festgeld_tagesgeld['date'] = pd.to_datetime(df_festgeld_tagesgeld['date'])

# Extract the year from the 'date' column
df_festgeld_tagesgeld['year'] = df_festgeld_tagesgeld['date'].dt.year

# Convert all columns except 'date' and 'year' to numeric, coercing errors to NaN
numeric_columns = df_festgeld_tagesgeld.columns.difference(['date', 'year'])
df_festgeld_tagesgeld[numeric_columns] = df_festgeld_tagesgeld[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Group by year and calculate the yearly values
def calculate_yearly_growth(group):
    results = {}
    for column in group.columns:
        count = group[column].count()
        if count == 0:  
            results[column] = np.nan
        else:
            results[column] = (np.prod(group[column].dropna() + 1) ** (1 / count)) - 1
    return pd.Series(results)

df_festgeld_tagesgeld_yearly = df_festgeld_tagesgeld.groupby('date')[numeric_columns].apply(calculate_yearly_growth).reset_index()

# Display the yearly values
df_festgeld_tagesgeld_yearly.head(10)

,date,Festgeld_05,Festgeld_1,Festgeld_10,Festgeld_2,Festgeld_5,Tagesgeld
0,1994-01-30,4.2765,4.2350,5.885000,4.947500,5.075000,NaN
1,1994-02-28,4.1880,4.2850,5.785714,4.875000,5.103906,NaN
2,1994-03-28,4.1375,4.3350,5.758929,4.864583,5.164927,NaN
3,1994-04-26,4.0925,4.4300,5.750000,4.852500,5.159844,NaN
4,1994-05-24,3.9150,4.3425,5.914062,4.848646,5.318677,NaN
5,1994-06-28,3.7450,4.1675,6.234375,4.814732,5.553333,NaN
6,1994-07-25,3.6525,4.1025,6.376116,4.880556,5.813235,NaN
7,1994-08-23,3.6775,4.1525,6.417500,4.950000,5.907500,NaN
8,1994-09-26,3.7600,4.2625,6.667500,5.172500,6.235000,NaN
9,1994-10-24,3.9000,4.4100,6.972500,5.312500,6.565000,NaN


## Restructuring

In [ ]:
# Define the range of years
year_range = range(1, 31)  

# Initialize the structure for the JSON
json_structure = {
    "assets": []
}

# Define the function to convert numpy types and pandas.Timestamp to native Python types for JSON serialization
def convert_numpy_types(obj):
    if isinstance(obj, (np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, pd.Timestamp):  
        return obj.strftime('%Y')  
    else:
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

# Function to process columns and add them to the JSON structure
def process_columns(df, category, date_column_name="date"):
    for column_name in df.columns:
        if column_name == date_column_name:
            continue  

        growth_column = df[column_name]
        date_column = df[date_column_name]

        # Determine the category dynamically for Festgeld and Tagesgeld
        if "Festgeld" in column_name:
            column_category = "Festgeld"
        elif "Tagesgeld" in column_name:
            column_category = "Tagesgeld"
        else:
            column_category = category  

        # Add a new asset entry for the current column
        asset_entry = {
            "category": column_category,
            "label": f"{column_name}",
            "kennung": f"{column_category} {column_name}",
            "id": f"{column_category}_{column_name}",
            "data": [] 
        }

        # Iterate through all values of year
        for year in year_range:
            
            highest_value = float('-inf')
            lowest_value = float('inf')
            highest_year = None
            lowest_year = None
            highest_values_list = []  
            lowest_values_list = [] 
            median_values_list = []  
            values_with_years = [] 

            # Iterate through all rows where there are numbers
            valid_growths = growth_column.dropna()
            valid_dates = date_column[~growth_column.isna()]  

            for i in range(len(valid_growths) - year + 1): 
                value = 1
                growth_array = [] 
                for growth in valid_growths[i:i + year]:
                    value *= (1 + growth / 100)  
                    growth_array.append(round(growth, 2))  
                
                values_with_years.append((value, valid_dates.iloc[i], growth_array))  
                if value > highest_value:
                    highest_value = value
                    highest_year = valid_dates.iloc[i]
                    highest_values_list = growth_array  
                if value < lowest_value:
                    lowest_value = value
                    lowest_year = valid_dates.iloc[i]
                    lowest_values_list = growth_array  

            # Calculate the median value and find its corresponding year and array
            if values_with_years:  
                sorted_values = sorted(values_with_years, key=lambda v: v[0])
                median_index = len(sorted_values) // 2
                median_value, median_year, median_values_list = sorted_values[median_index]

                # Create dictionaries for max, median, and min arrays
                max_dict = {
                    "type": "max",
                    "duration": year, 
                    "year": highest_year,  
                    "growth_array": highest_values_list
                }
                median_dict = {
                    "type": "median",
                    "duration": year,  
                    "year": median_year, 
                    "growth_array": median_values_list
                }
                min_dict = {
                    "type": "min",
                    "duration": year, 
                    "year": lowest_year, 
                    "growth_array": lowest_values_list
                }

                # Append the dictionaries to the "data" key in the asset entry
                asset_entry["data"].extend([max_dict, median_dict, min_dict])
            else:
                print(f"No valid values for column '{column_name}' and year = {year}. Skipping...")

        # Append the asset entry to the JSON structure
        json_structure["assets"].append(asset_entry)

# Reset the index to bring 'date' back as a column in df_anleihen_clean
df_anleihen_inflation.reset_index(inplace=True)

# Ensure the 'date' column exists in df_festgeld_tagesgeld_yearly
if 'date' not in df_festgeld_tagesgeld_yearly.columns:
    df_festgeld_tagesgeld_yearly.reset_index(inplace=True)

# Process df_anleihen_clean, df_etf, and df_festgeld_tagesgeld_yearly
process_columns(df_anleihen_inflation, "Anleihe", date_column_name="date")
process_columns(df_etf, "Index", date_column_name="date")
process_columns(df_festgeld_tagesgeld_yearly, "Festgeld_Tagesgeld", date_column_name="date")

# Save the JSON structure to a file
with open("export/data.json", "w") as json_file:
    json.dump(json_structure, json_file, separators=(",", ":"), default=convert_numpy_types)

No valid values for column 'Australien' and year = 1. Skipping...
No valid values for column 'Australien' and year = 2. Skipping...
No valid values for column 'Australien' and year = 3. Skipping...
No valid values for column 'Australien' and year = 4. Skipping...
No valid values for column 'Australien' and year = 5. Skipping...
No valid values for column 'Australien' and year = 6. Skipping...
No valid values for column 'Australien' and year = 7. Skipping...
No valid values for column 'Australien' and year = 8. Skipping...
No valid values for column 'Australien' and year = 9. Skipping...
No valid values for column 'Australien' and year = 10. Skipping...
No valid values for column 'Australien' and year = 11. Skipping...
No valid values for column 'Australien' and year = 12. Skipping...
No valid values for column 'Australien' and year = 13. Skipping...
No valid values for column 'Australien' and year = 14. Skipping...
No valid values for column 'Australien' and year = 15. Skipping...
No v